In [68]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [69]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.15  # Reserve 15% of the data for validation
)

In [70]:
train_generator = train_datagen.flow_from_directory(
    "C:\projects\data",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


Found 1830 images belonging to 3 classes.


In [71]:
validation_generator = train_datagen.flow_from_directory(
    "C:\projects\data",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 322 images belonging to 3 classes.


In [72]:
# Load pre-trained VGG16 model without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Define the number of classes based on your dataset
num_classes = len(train_generator.class_indices)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [73]:
# Combine base model with custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [74]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10
57/57 [==============================] - 212s 4s/step - loss: 0.6092 - accuracy: 0.7347 - val_loss: 0.3964 - val_accuracy: 0.8469
Epoch 2/10
57/57 [==============================] - 147s 3s/step - loss: 0.3682 - accuracy: 0.8548 - val_loss: 0.2841 - val_accuracy: 0.9000
Epoch 3/10
57/57 [==============================] - 148s 3s/step - loss: 0.2790 - accuracy: 0.9010 - val_loss: 0.2223 - val_accuracy: 0.9250
Epoch 4/10
57/57 [==============================] - 170s 3s/step - loss: 0.2592 - accuracy: 0.8993 - val_loss: 0.2850 - val_accuracy: 0.8750
Epoch 5/10
57/57 [==============================] - 181s 3s/step - loss: 0.2263 - accuracy: 0.9138 - val_loss: 0.1632 - val_accuracy: 0.9625
Epoch 6/10
57/57 [==============================] - 182s 3s/step - loss: 0.2058 - accuracy: 0.9238 - val_loss: 0.2112 - val_accuracy: 0.9156
Epoch 7/10
57/57 [==============================] - 190s 3s/step - loss: 0.2033 - accuracy: 0.9194 - val_loss: 0.2196 - val_accuracy: 0.8969
Epoch 8/10
57

In [75]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print("Validation Accuracy: {:.2f}%".format(accuracy * 100))

11/11 [==============================] - 23s 2s/step - loss: 0.1594 - accuracy: 0.9348
Validation Accuracy: 93.48%


In [76]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(validation_generator)
print("Validation Accuracy: {:.2f}%".format(accuracy * 100))


11/11 [==============================] - 24s 2s/step - loss: 0.1443 - accuracy: 0.9441
Validation Accuracy: 94.41%


In [77]:
# Generate predictions on the validation set
validation_predictions = model.predict(validation_generator)
predicted_classes = np.argmax(validation_predictions, axis=1)
true_classes = validation_generator.classes


11/11 [==============================] - 23s 2s/step


In [78]:
# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[68 70 12]
 [68 74  8]
 [ 9 11  2]]


In [79]:
# Compute classification report
class_names = list(validation_generator.class_indices.keys())
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_names))


Classification Report:
                       precision    recall  f1-score   support

Potato___Early_blight       0.47      0.45      0.46       150
 Potato___Late_blight       0.48      0.49      0.49       150
     Potato___healthy       0.09      0.09      0.09        22

             accuracy                           0.45       322
            macro avg       0.35      0.35      0.35       322
         weighted avg       0.45      0.45      0.45       322



In [92]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the uploaded image
img_path = r"C:\projects\scrapping\model\Data\download.webp" # Update with the path to your uploaded image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalize pixel values


In [93]:
# Make predictions
predictions = model.predict(img_array)

# Interpret results
class_names = list(train_generator.class_indices.keys())
predicted_class_index = np.argmax(predictions)
predicted_class = class_names[predicted_class_index]
confidence = predictions[0][predicted_class_index]

print("Predicted Class:", predicted_class)
print("Confidence:", confidence)

1/1 [==============================] - 0s 98ms/step
Predicted Class: Potato___healthy
Confidence: 0.9511755


In [94]:
model.save('potato_leaf_model.h5')